# Edge cases for load

In [ ]:
import lndb_setup as lndb
from lndb_setup import settings
from lndb_setup._clone import setup_local_test_postgres
from lndb_setup._settings_store import instance_settings_file
import shutil
import pytest

## Load by instance name

If the user is the instance owner, just load the instance by name:

```
!lndb load mydata
```

In [ ]:
lndb.load("mydata")

In [ ]:
import os

assert settings.instance.storage.is_cloud == False
assert settings.instance.name == "mydata"
assert settings.instance.storage.root.as_posix() == f"{os.getcwd()}/mydata"
assert settings.instance.storage.cache_dir is None
assert settings.instance.db == f"sqlite:///{os.getcwd()}/mydata/mydata.lndb"

In [ ]:
# another test case, this time with a manually configured instance name

import os

lndb.load("mydata_instance_name")
assert settings.instance.storage.is_cloud == False
assert settings.instance.name == "mydata_instance_name"
assert settings.instance.storage.root.as_posix() == f"{os.getcwd()}/mydata_storage"
assert settings.instance.storage.cache_dir is None
assert (
    settings.instance.db
    == f"sqlite:///{os.getcwd()}/mydata_storage/mydata_instance_name.lndb"
)
lndb.delete("mydata_instance_name")

This also works for remote instances:

In [ ]:
# ensure that the locally cached env file is deleted
instance_settings_file("lndb-setup-ci", "testuser1").unlink()

In [ ]:
lndb.load("lndb-setup-ci")  # lndb load lndb-setup-ci

In [ ]:
assert settings.instance.storage.is_cloud == True
assert str(settings.instance.storage.root) == "s3://lndb-setup-ci"
assert str(settings.instance._sqlite_file) == "s3://lndb-setup-ci/lndb-setup-ci.lndb"

# clean up
settings.instance._sqlite_file.unlink()
lndb.delete("lndb-setup-ci")

## Load instance from another owner

If you have the permission, you can load an instance from another owner. 

In [ ]:
load("testuser1/mydata")  # lndb load "testuser1/mydata"

Load an instance from wrong owner.

In [ ]:
assert load("testuser2/mydata") == "instance-not-exists"

## Local instances can not be loaded via the hub

It's not possible to load an instance from the hub if it has a local component, say, local storage.

In [ ]:
init(storage="local_storage_instance")

In [ ]:
instance_settings_file("local_storage_instance", "testuser1").unlink()

In [ ]:
assert load("local_storage_instance") == "instance-not-exists"

It's not possible to load an instance from the hub if it's using a local db.

In [ ]:
pgurl = setup_local_test_postgres("pgtest_remote_storage")

In [ ]:
init(storage="s3://lndb-setup-ci-pgtest-remote", db=pgurl, name="pgtest_remote_storage")
instance_settings_file("pgtest_remote_storage", "testuser1").unlink()

In [ ]:
assert load("pgtest_remote_storage") == "instance-not-exists"

In [ ]:
!docker stop pgtest_remote_storage && docker rm pgtest_remote_storage